In [ ]:
pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 28.3 MB/s eta 0:00:00


In [ ]:
pip install -q google-colab-selenium

In [ ]:
import pandas as pd
import numpy as np
import os
from ipywidgets import IntSlider
import plotly.express as px
from datetime import date
import requests
import json
from pandas import json_normalize
pd.set_option('display.max_columns', None)

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from google.colab import files


In [ ]:
import google_colab_selenium as gs
driver = gs.Chrome()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
###enter your urls here
review_urls = """


"""

In [ ]:
url_list = [a for a in review_urls.split("\n") if len(a) > 20]
url_list

['https://www.trendyol.com/dermoskin/be-bright-arbutin-kompleks-krem-33-ml-p-4363619?boutiqueId=61&merchantId=694902',
 'https://www.trendyol.com/dermoskin/be-bright-pigmentyl-spf-50-gunes-koruyucu-krem-75-ml-2-li-paket-p-7115213?boutiqueId=61&merchantId=838714']

In [ ]:
driver.maximize_window()

In [ ]:
driver.get(url_list[0])

In [ ]:
# accepting cookies
try:
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID,'onetrust-accept-btn-handler'))).click()
    print("cookies accepted")
except:
    print("cookies cannot accepted")

cookies accepted


In [ ]:
def go_revs():
  try:
      navigate_button = WebDriverWait(driver, 10).until(
          EC.element_to_be_clickable((By.CLASS_NAME, 'navigate-all-reviews-btn'))
      )
      navigate_button.click()
      print("Butona başarıyla tıklandı.")
  except:
      print("Butona tıklama sırasında bir hata oluştu:", )

go_revs()

Butona başarıyla tıklandı.


In [ ]:
# clicking somewhere blank
from selenium.webdriver.common.action_chains import ActionChains
actions = ActionChains(driver)
actions.move_by_offset(10, 10).click().perform()


In [ ]:
def select_new_to_old():

  dropdown_button = driver.find_element(By.CLASS_NAME, "selected-container")
  dropdown_button.click()

  yeniden_eskiye_option = driver.find_element(By.XPATH, "//li[text()='Yeniden Eskiye']")
  yeniden_eskiye_option.click()

  time.sleep(2)
select_new_to_old()

In [ ]:
categories = driver.find_elements(By.XPATH, "//div[@data-testid='sliderList']//a[contains(@class, 'ps-tags__tag')]")

In [ ]:
def scroll(scroll_count):
  for i in range(scroll_count):
      total_height = driver.execute_script("return document.body.scrollHeight")
      scroll_to =str(int(round( total_height * 0.75)))
      driver.execute_script(f"window.scrollTo(0, {scroll_to});")
      time.sleep(2)

def scrape(url):
  global dd_list
  driver.get(url)
  print("url got")
  title = driver.title
  print(title)
  go_revs()
  print("go reviews")
  select_new_to_old()
  print("select new to old")
  categories = driver.find_elements(By.XPATH, "//div[@data-testid='sliderList']//a[contains(@class, 'ps-tags__tag')]")
  print("caterories got")

  category_dic = {}
  dd_list = []
  for category_idx in [a for a in range(len(categories)) if a != 1]:
          categories = driver.find_elements(By.XPATH, "//div[@data-testid='sliderList']//a[contains(@class, 'ps-tags__tag')]")
          category = categories[category_idx]
          category.click()
          print(f"cat {category} clicked")

          full_text = category.text.strip()
          category_name = category.find_element(By.CLASS_NAME, "ps-tags__tag--bold").text.strip()
          print(category_name)

          cat_review_count = full_text.replace(category_name, "").strip("()")

          cat_review_count = cat_review_count.replace("\n(","")

          time.sleep(2)

          stars = driver.find_elements(By.XPATH, "//div[contains(@class, 'ps-stars__content')]")
          print("stars found")
          star_dic = {}
          cc_list = []
          for star in stars:
              full_stars = star.find_elements(By.CSS_SELECTOR, ".star-w .full[style='width: 100%; max-width: 100%;']")
              num_full_stars = len(full_stars)
              rating_count = star.find_element(By.CLASS_NAME, 'ps-stars__count').text.strip("()")
              print("inside stars")
              star.click()
              time.sleep(2)
              if int(rating_count)>250:
                  scroll(5)
              elif int(rating_count) <= 250 and int(rating_count) > 150:
                  scroll(4)
              elif int(rating_count) <= 150 and int(rating_count) > 70:
                  scroll(3)
              elif int(rating_count) <= 70 and int(rating_count) > 30:
                  scroll(1)

              comment_dic = {}
              comment_list = []
              commenter_list = []
              comment_date_list = []
              seller_name_list = []

              comments = driver.find_elements(By.CLASS_NAME, "comment")
              print("inside comment")
              for comment in comments:
                  comment_text = comment.find_element(By.CLASS_NAME, "comment-text").text.strip()
                  comment_info_items = comment.find_elements(By.CLASS_NAME, "comment-info-item")
                  commenter = comment_info_items[0].text.strip() if len(comment_info_items) > 0 else "Bilgi yok"
                  comment_date = comment_info_items[1].text.strip() if len(comment_info_items) > 1 else "Tarih bilgisi yok"

                  seller_info = comment.find_element(By.CLASS_NAME, "seller-info")
                  seller_name = seller_info.text.strip()
                  comment_list.append(comment_text)
                  commenter_list.append(commenter)
                  comment_date_list.append(comment_date)
                  seller_name_list.append(seller_name)


              comment_dic["comment_text"] = comment_list
              comment_dic["commenter"] = commenter_list
              comment_dic["comment_date"] = comment_date_list
              comment_dic["seller_name"] = seller_name_list
              comm = []
              for i in comment_dic.keys():
                  comm.append(pd.Series(comment_dic[i], name=i))
              comment_df = pd.concat(comm,axis=1)
              star.click()
              time.sleep(3)
              comment_df["rating"] = num_full_stars
              comment_df["rating_count"] = rating_count
              cc_list.append(comment_df)

          print("stars finished")
          ww = pd.concat(cc_list)
          ww["category_name"] = category_name
          ww["cat_review_count"] = cat_review_count

          dd_list.append(ww)
          time.sleep(4)

  fin_df = pd.concat(dd_list)
  fin_df["title"] = title
  fin_df["url"] = url
  return fin_df, title

In [ ]:
for url in url_list:
  df, title = scrape(url)
  df.to_excel(f"{title}_comments.xlsx")
  files.download(f"{title}_comments.xlsx")




In [ ]:
driver.quit()